In [1]:
def sha256_padding(message):
    # Step 1: Add a single "1" bit to the end of the message
    padded_message = message + b'\x80'

    # Step 2: Append "0" bits until the length is congruent to 448 bits modulo 512 bits
    while len(padded_message) % 64 != 56:
        padded_message += b'\x00'

    # Step 3: Append the 64-bit representation of the original message length
    message_bit_length = len(message) * 8  # Length of original message in bits
    padded_message += message_bit_length.to_bytes(8, byteorder='big')

    return padded_message

# Example usage:
message = b'Hello, world!'
padded_message = sha256_padding(message)
print("Padded Message (in bytes):", padded_message)
print("Padded Message (in hexadecimal):", padded_message.hex())


Padded Message (in bytes): b'Hello, world!\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00h'
Padded Message (in hexadecimal): 48656c6c6f2c20776f726c6421800000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000068


In [5]:
message_bit_length = len(message) * 8
message_bit_length, len(message), message_bit_length.to_bytes(8, byteorder='big')

(104, 13, b'\x00\x00\x00\x00\x00\x00\x00h')

In [6]:
len(message_bit_length.to_bytes(8, byteorder='big'))

8

In [8]:
def parse_message_blocks(padded_message):
    # Determine the number of blocks
    num_blocks = len(padded_message) // 64

    # Parse the padded message into blocks of 512 bits (64 bytes) each
    blocks = [padded_message[i*64:(i+1)*64] for i in range(num_blocks)]

    return blocks

# Example usage:
padded_message = sha256_padding(message)  # Assume the message has already been padded
message_blocks = parse_message_blocks(padded_message)
print("Number of blocks:", len(message_blocks))
for i, block in enumerate(message_blocks):
    print(f"Block {i + 1}: {block}")

Number of blocks: 1
Block 1: b'Hello, world!\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00h'


In [14]:

def initial_hash_value(prime_number_index):
    # First 32 bits of the fractional part of the square root of the prime number
    fractional_part = math.sqrt(prime_number_index)
    fractional_part -= int(fractional_part)  # Remove the integer part
    fractional_part *= 2**32  # Convert fractional part to a 32-bit value
    return int(fractional_part)

# First 8 prime numbers
prime_numbers = [2, 3, 5, 7, 11, 13, 17, 19]

# Calculate the initial hash value using the first prime number (index 0)
initial_value = initial_hash_value(prime_numbers[0])
print("Initial hash value:", hex(initial_value))

Initial hash value: 0x6a09e667


In [9]:
import math

def initialize_hash_values():
    # First 32 bits of the fractional parts of the square roots of the first 8 prime numbers
    initial_hash_values = [
        0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,
        0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19
    ]
    return initial_hash_values

# Example usage:
hash_values = initialize_hash_values()
print("Initial hash values:", hash_values)

Initial hash values: [1779033703, 3144134277, 1013904242, 2773480762, 1359893119, 2600822924, 528734635, 1541459225]


In [13]:
import numpy as np
np.sqrt(2)

1.4142135623730951

In [15]:
def generate_message_schedule(block):
    # Initialize the message schedule array with the first 16 words of the block
    message_schedule = [int.from_bytes(block[i:i+4], byteorder='big') for i in range(0, 64, 4)]

    # Extend the message schedule array to contain 64 words
    for i in range(16, 64):
        s0 = (right_rotate(message_schedule[i-15], 7) ^
              right_rotate(message_schedule[i-15], 18) ^
              (message_schedule[i-15] >> 3))
        s1 = (right_rotate(message_schedule[i-2], 17) ^
              right_rotate(message_schedule[i-2], 19) ^
              (message_schedule[i-2] >> 10))
        message_schedule.append((message_schedule[i-16] + s0 + message_schedule[i-7] + s1) & 0xFFFFFFFF)

    return message_schedule

# Function for right rotation (circular shift) of a 32-bit integer
def right_rotate(x, n):
    return ((x >> n) | (x << (32 - n))) & 0xFFFFFFFF

# Example usage:
block = b'abcdefghabcdefghabcdefghabcdefghabcdefghabcdefghabcdefghabcdefgh'
message_schedule = generate_message_schedule(block)
print("Message schedule array:", message_schedule)


Message schedule array: [1633837924, 1701209960, 1633837924, 1701209960, 1633837924, 1701209960, 1633837924, 1701209960, 1633837924, 1701209960, 1633837924, 1701209960, 1633837924, 1701209960, 1633837924, 1701209960, 2320222283, 574669636, 2844986084, 2629984534, 4035545471, 2676951327, 36767869, 2977276057, 1854991678, 3483678522, 2936290453, 2138134756, 3768658282, 1262513367, 171188613, 2217344388, 2881403479, 2976990708, 1513398942, 2340158310, 4015880860, 3843913827, 3857413675, 3128544316, 932287527, 992776706, 3292833739, 3450750126, 1236765923, 525428467, 1671027472, 1880165015, 1812991627, 957393826, 1360195865, 2423313978, 3134985313, 1370593993, 4223910515, 1380492695, 830404577, 2040545542, 3468396201, 3722789355, 193192414, 2122655158, 2768274483, 4043001627]


In [18]:
x = 16     # binary: 00001000
result = x >> 2   # shift x right by 2 positions
print(result)   

4
